In [178]:
import pandas as pd
import yfinance as yf
import re
import numpy as np


def get_currency(x):
    if pd.notna(x) and x != "":
        return yf.Ticker(x).info["currency"]
    else:
        return ""

class Portfolio:
    def __init__(self, path="portfolio.csv",):
        self.path = path
        self.df_raw = pd.read_csv(path, usecols=["Investment","Quantity"], index_col=False)
        self.usd_gbp = yf.Ticker("USDGBP=X")
        self.cash = {"GBP": self.df_raw.iloc[-1]["Quantity"],
                     "USD": round(self.df_raw.iloc[-1]["Quantity"] * (1 / self.usd_gbp.info["bid"]), 2)}


    def __repr__(self):
        return self.df_stocks.to_string()
    

    def _init_stocks_df(self):
        self.df_stocks = self.df_raw.copy().iloc[0:-1]

        self.df_stocks["Exchange"] = self.df_stocks["Investment"].apply(lambda x: re.search(r"\((.*?)\:", x).group()[1:-1] if re.search(r"\((.*?)\:", x) else "")
        self.df_stocks["Ticker"] = self.df_stocks["Investment"].apply(lambda x: re.search(r"\:(.*?)\)", x).group()[1:-1] if re.search(r"\((.*?)\:", x) else "")
        self.df_stocks["Ticker"] = self.df_stocks["Ticker"] + self.df_stocks["Exchange"].apply(lambda x: ".L" if x == "LSE" else "")
        
        self.df_stocks["Currency"] = ""
        self.df_stocks["Currency"] = self.df_stocks["Ticker"].where(self.df_stocks["Ticker"] != "").apply(get_currency)
        self.df_stocks["Price Raw"] = self.df_stocks["Ticker"].apply(lambda x: yf.Ticker(x).info["bid"])
        
        self.df_stocks["Price GBP"] = np.nan
        self.df_stocks["Price GBP"] = self.df_stocks["Price GBP"].where(self.df_stocks["Currency"] != "GBP", self.df_stocks["Price Raw"]).round(2)
        self.df_stocks["Price GBP"] = self.df_stocks["Price GBP"].where(self.df_stocks["Currency"] != "USD", self.df_stocks["Price Raw"] * self.usd_gbp.info["bid"]).round(2)
        self.df_stocks["Price GBP"] = self.df_stocks["Price GBP"].where(self.df_stocks["Currency"] != "GBp", self.df_stocks["Price Raw"] / 100).round(2)
    
        self.df_stocks["Price USD"] = (self.df_stocks["Price GBP"] * (1 / self.usd_gbp.info["bid"])).round(2)

        self.df_stocks["Total Value GBP"] = self.df_stocks["Price GBP"] * self.df_stocks["Quantity"]
        self.df_stocks["Total Value USD"] = self.df_stocks["Price USD"] * self.df_stocks["Quantity"]


    def show_portoflio_totals(self):
        print("Portfolio:")
        print(f"£{self.df_stocks['Total Value GBP'].sum() + self.cash['GBP']:.2f}")
        print(f"${self.df_stocks['Total Value USD'].sum() + self.cash['USD']:.2f}")


portfolio = Portfolio()
portfolio._init_stocks_df()
portfolio.show_portoflio_totals()

Portfolio:
£38181.25
$48223.77


In [106]:


# x = yf.Ticker("DJEL.L")
# x = yf.Ticker("USDGBP=X")

# print(x.info["bid"])

# for i, value in enumerate(x.info.items()):
#     print(i, value)

print(yf.Ticker("").info)



404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=&crumb=%7B%22finance%22%3A%7B%22result%22%3Anull%2C%22error%22%3A%7B%22code%22%3A%22Unauthorized%22%2C%22description%22%3A%22Invalid+Cookie%22%7D%7D%7D


YFinanceException: Failed to parse json response from Yahoo Finance: {'code': 'Not Found', 'description': 'HTTP 404 Not Found'}